## EDA: Clustering based on ARMA-Garch model features + Model Comparison on each cluster
- ### Models tested were ARMA - Garch and AR - EWMA
- ### Both AREWMA (Autoregressive Exponentially Weighted Moving Average) and ARMA-GARCH are traditional statistical models used for time series analysis   

#### The code below is to understand what columns are in the parquet files prints out first few lines of a random parquet file, imports are done seperately because each code block was done at seperate times.

In [ ]:
import pandas as pd
import pyarrow.parquet as pq
import os

# Directory containing your parquet files
parquet_dir = "."  # Change this to your parquet directory path if different

# Get a list of all parquet files in the directory
parquet_files = [f for f in os.listdir(parquet_dir) if f.endswith('.parquet')]

# Display the first few rows of the first three files
for i, file in enumerate(parquet_files[:3]):
    file_path = os.path.join(parquet_dir, file)
    print(f"\n{'='*50}")
    print(f"File {i+1}: {file}")
    print(f"{'='*50}")
    
    # Read the parquet file
    df = pd.read_parquet(file_path)
    
    # Display basic info
    print(f"Shape: {df.shape}")
    print(f"Columns: {df.columns.tolist()}")
    print("\nFirst 5 rows:")
    print(df.head())
        


File 1: stock_94.parquet
Shape: (1191489, 16)
Columns: ['time_id', 'seconds_in_bucket', 'bid_price1', 'ask_price1', 'bid_price2', 'ask_price2', 'bid_size1', 'ask_size1', 'bid_size2', 'ask_size2', 'stock_id', 'WAP', 'BidAskSpread', 'log_return', 'time_bucket', 'volatility']

First 5 rows:
   time_id  seconds_in_bucket  bid_price1  ask_price1  bid_price2  ask_price2  \
0        5                  0    1.000301    1.000903    1.000129    1.000989   
1        5                  3    1.000301    1.000903    1.000129    1.000989   
2        5                  4    1.000301    1.000903    1.000043    1.000989   
3        5                  5    1.000301    1.000817    1.000043    1.000903   
4        5                  6    1.000301    1.000817    1.000043    1.000903   

   bid_size1  ask_size1  bid_size2  ask_size2  stock_id       WAP  \
0        101         60        101         10        94  1.000679   
1        101         60        101         10        94  1.000679   
2        101    

#### The code below is to understand the number of unique time-ids and its distribution
#### Results:
- #### 3830 unique time ids in stock 94

In [ ]:
import pandas as pd
import os

# Path to one of your parquet files
file_path = "stock_1.parquet"  # Adjust this path

# Read the parquet file
df = pd.read_parquet(file_path)

# Get unique time_ids and sort them
unique_time_ids = sorted(df['time_id'].unique())

# Print the number of unique time_ids
print(f"Number of unique time_ids: {len(unique_time_ids)}")

# Print the first 20 unique time_ids (to avoid printing too many)
print(f"First 20 unique time_ids: {unique_time_ids[:20]}")

# Get the distribution/count of rows for each time_id
time_id_counts = df['time_id'].value_counts().sort_index()

# Print some statistics about the distribution
print(f"\nMin rows per time_id: {time_id_counts.min()}")
print(f"Max rows per time_id: {time_id_counts.max()}")
print(f"Average rows per time_id: {time_id_counts.mean():.2f}")

# Print the distribution for the first few time_ids
print("\nSample distribution (rows per time_id):")
print(time_id_counts.head(10))

Number of unique time_ids: 3830
First 20 unique time_ids: [5, 11, 16, 31, 62, 72, 97, 103, 109, 123, 128, 146, 147, 152, 157, 159, 169, 207, 211, 213]

Min rows per time_id: 132
Max rows per time_id: 599
Average rows per time_id: 393.61

Sample distribution (rows per time_id):
time_id
5      575
11     370
16     353
31     171
62     227
72     514
97     421
103    487
109    564
123    516
Name: count, dtype: int64


#### This code is to select 100 out of 3830 time-ids 500 time ids were not selected because other operations (see next code block) were too computationally intensive (took too long to run my laptop ran out of application memory)

In [ ]:
import pandas as pd
import numpy as np

# Path to one of your parquet files
file_path = "stock_1.parquet"  # Adjust this path

# Read the parquet file
df = pd.read_parquet(file_path)

# Get unique time_ids and sort them
unique_time_ids = sorted(df['time_id'].unique())

# Get first, middle, and last time_ids
first_time_id = unique_time_ids[0]
middle_time_id = unique_time_ids[len(unique_time_ids)//2]
last_time_id = unique_time_ids[-1]

print(f"First time_id: {first_time_id}")
print(f"Middle time_id: {middle_time_id}")
print(f"Last time_id: {last_time_id}")

# Select a total of 500 time_ids spread across the dataset
num_samples = 100

# Check if we have enough unique time_ids
if len(unique_time_ids) <= num_samples:
    selected_time_ids = unique_time_ids
    print(f"\nUsing all {len(unique_time_ids)} available time_ids:")
else:
    # Use linspace to get evenly distributed indices
    selected_indices = np.linspace(0, len(unique_time_ids)-1, num_samples, dtype=int)
    selected_time_ids = [unique_time_ids[i] for i in selected_indices]
    print(f"\nSelected {num_samples} time_ids:")

print(selected_time_ids)

First time_id: 5
Middle time_id: 15853
Last time_id: 32767

Selected 100 time_ids:
[5, 335, 697, 1107, 1326, 1640, 1938, 2178, 2479, 2833, 3113, 3469, 3877, 4138, 4460, 4743, 5101, 5407, 5729, 5978, 6325, 6643, 6930, 7245, 7629, 7921, 8441, 8718, 9091, 9367, 9819, 10100, 10491, 10781, 11070, 11431, 11764, 12030, 12348, 12661, 12965, 13318, 13579, 13868, 14176, 14452, 14795, 15038, 15418, 15728, 15989, 16404, 16704, 17029, 17305, 17606, 17914, 18184, 18502, 18836, 19157, 19488, 19747, 20135, 20460, 20827, 21145, 21507, 21891, 22249, 22622, 22922, 23220, 23531, 23863, 24224, 24707, 25059, 25347, 25636, 25924, 26307, 26603, 26944, 27240, 27595, 27964, 28379, 28801, 29171, 29590, 29941, 30313, 30624, 31047, 31322, 31609, 32083, 32463, 32767]


#### The code below filters out 100 time ids that were selected, performs feature calculation and stores them for clustering.

In [ ]:
import pandas as pd
import numpy as np
import os
import glob
from sklearn.cluster import KMeans
from statsmodels.stats.diagnostic import acorr_ljungbox
from statsmodels.tsa.stattools import adfuller, acf
import warnings
warnings.filterwarnings('ignore')

# Directory containing parquet files
parquet_dir = "."  # Change this to your directory

# Use the time_ids we identified
selected_time_ids = [5, 335, 697, 1107, 1326, 1640, 1938, 2178, 2479, 2833, 3113, 3469, 3877, 4138, 4460, 4743, 5101, 5407, 5729, 5978, 6325, 6643, 6930, 7245, 7629, 7921, 8441, 8718, 9091, 9367, 9819, 10100, 10491, 10781, 11070, 11431, 11764, 12030, 12348, 12661, 12965, 13318, 13579, 13868, 14176, 14452, 14795, 15038, 15418, 15728, 15989, 16404, 16704, 17029, 17305, 17606, 17914, 18184, 18502, 18836, 19157, 19488, 19747, 20135, 20460, 20827, 21145, 21507, 21891, 22249, 22622, 22922, 23220, 23531, 23863, 24224, 24707, 25059, 25347, 25636, 25924, 26307, 26603, 26944, 27240, 27595, 27964, 28379, 28801, 29171, 29590, 29941, 30313, 30624, 31047, 31322, 31609, 32083, 32463, 32767]
# Dictionary to store ARMA-GARCH relevant features for each stock
stock_features = {}

# Find all parquet files in the directory
parquet_files = glob.glob(os.path.join(parquet_dir, "stock_*.parquet"))

for file_path in parquet_files:
    # Extract stock_id from filename
    stock_id = os.path.basename(file_path).split("_")[1].split(".")[0]
    print(f"Processing stock {stock_id}...")
    
    try:
        # Read the parquet file
        df = pd.read_parquet(file_path)
        
        # Filter for selected time_ids only
        subset = df[df['time_id'].isin(selected_time_ids)]
        
        if subset.empty or len(subset) < 100:  # Need sufficient data
            print(f"  Warning: Insufficient data for stock {stock_id}")
            continue
        
        # Sort by time_id and seconds_in_bucket to ensure time order
        subset = subset.sort_values(['time_id', 'seconds_in_bucket'])
        
        # Focus on log returns for ARMA-GARCH analysis
        returns = subset['log_return'].dropna()
        
        if len(returns) < 100:  # Need sufficient non-NA returns
            print(f"  Warning: Insufficient return data for stock {stock_id}")
            continue
            
        # ARMA relevant features
        
        # 1. Test for serial correlation (Ljung-Box test)
        lb_test = acorr_ljungbox(returns, lags=[10], return_df=True)
        serial_corr_pvalue = lb_test['lb_pvalue'].values[0]
        
        # 2. Calculate autocorrelation at different lags
        acf_values = acf(returns, nlags=10)
        
        # 3. Test for stationarity (Augmented Dickey-Fuller test)
        adf_result = adfuller(returns)
        adf_pvalue = adf_result[1]
        
        # GARCH relevant features
        
        # 1. Volatility clustering - autocorrelation in squared returns
        squared_returns = returns**2
        sq_acf_values = acf(squared_returns, nlags=10)
        
        # 2. Test for ARCH effects (Ljung-Box test on squared returns)
        arch_test = acorr_ljungbox(squared_returns, lags=[10], return_df=True)
        arch_effect_pvalue = arch_test['lb_pvalue'].values[0]
        
        # 3. Calculate kurtosis (excess kurtosis suggests fat tails)
        excess_kurtosis = returns.kurtosis()
        
        # 4. Calculate volatility of volatility (standard deviation of rolling volatility)
        rolling_vol = returns.rolling(window=20).std().dropna()
        vol_of_vol = rolling_vol.std()
        
        # Store features
        stock_features[stock_id] = {
            'stock_id': int(stock_id),
            # ARMA features
            'serial_corr_pvalue': serial_corr_pvalue,  # Lower suggests ARMA suitability
            'acf_lag1': acf_values[1],  # First-order autocorrelation
            'acf_lag5': acf_values[5],  # 5th-order autocorrelation
            'adf_pvalue': adf_pvalue,  # Lower suggests stationarity
            # GARCH features
            'arch_effect_pvalue': arch_effect_pvalue,  # Lower suggests GARCH suitability
            'sq_acf_lag1': sq_acf_values[1],  # Volatility clustering
            'sq_acf_lag5': sq_acf_values[5],  # Volatility persistence
            'excess_kurtosis': excess_kurtosis,  # Higher suggests fat tails
            'vol_of_vol': vol_of_vol,  # Volatility of volatility
            # Basic statistics
            'mean_return': returns.mean(),
            'std_return': returns.std(),
            'mean_spread': subset['BidAskSpread'].mean(),
            'observations': len(returns)
        }
        
    except Exception as e:
        print(f"  Error processing stock {stock_id}: {e}")

# Convert to DataFrame for clustering
clustering_df = pd.DataFrame.from_dict(stock_features, orient='index')

# Check if we have data
if clustering_df.empty:
    print("No data could be processed. Check your selected time_ids.")
else:
    print(f"Created clustering dataframe with shape: {clustering_df.shape}")
    
    # Save this DataFrame for later use
    clustering_df.to_csv("arma_garch_features.csv")
    
    print("Completed creating features for ARMA-GARCH clustering")
    
    # Display a sample of the data
    print("\nSample of the clustering dataframe:")
    print(clustering_df.head())

Processing stock 94...
Processing stock 84...
Processing stock 103...
Processing stock 113...


#### The code below just takes the time ids that were stored and performs clustering based on arma garch features and produces visualisations.

In [1]:
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import numpy as np
import pandas as pd

# Assuming clustering_df is your dataframe with ARMA-GARCH features
# If you're running this separately, load the saved CSV:
clustering_df = pd.read_csv("arma_garch_features.csv")

# Save stock_ids and drop from clustering features
stock_ids = clustering_df['stock_id'].copy()
features_for_clustering = clustering_df.drop('stock_id', axis=1).copy()

# Handle any remaining NaN values
features_for_clustering = features_for_clustering.fillna(0)

# Scale the features for clustering and visualization
scaler = StandardScaler()
scaled_features = scaler.fit_transform(features_for_clustering)
scaled_df = pd.DataFrame(scaled_features, columns=features_for_clustering.columns)

# Perform K-means clustering if not done already
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

# Find optimal number of clusters
print("\n--- FINDING OPTIMAL NUMBER OF CLUSTERS ---")
silhouette_scores = []
k_range = range(2, 11)
for k in k_range:
    kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
    cluster_labels = kmeans.fit_predict(scaled_features)
    silhouette_avg = silhouette_score(scaled_features, cluster_labels)
    silhouette_scores.append(silhouette_avg)
    print(f"For n_clusters = {k}, the silhouette score is {silhouette_avg}")

# Use optimal number of clusters
best_k = k_range[np.argmax(silhouette_scores)]
print(f"\nBest number of clusters: {best_k}")

# Final clustering with best_k
kmeans = KMeans(n_clusters=best_k, random_state=42, n_init=10)
cluster_labels = kmeans.fit_predict(scaled_features)

# Add cluster assignments back to the dataframe
clustering_df['cluster'] = cluster_labels

# Print cluster distribution
print("\n--- CLUSTER DISTRIBUTION ---")
cluster_counts = clustering_df['cluster'].value_counts().sort_index()
for cluster, count in cluster_counts.items():
    print(f"Cluster {cluster}: {count} stocks")

# Create visualizations
print("\n--- CREATING VISUALIZATIONS ---")

# 1. Silhouette score plot
print("Generating silhouette score plot...")
plt.figure(figsize=(10, 6))
plt.plot(k_range, silhouette_scores, 'o-', color='#3498db', linewidth=2, markersize=8)
plt.grid(True, linestyle='--', alpha=0.7)
plt.xlabel('Number of Clusters', fontsize=12)
plt.ylabel('Silhouette Score', fontsize=12)
plt.title('Optimal Number of Clusters', fontsize=14)
plt.xticks(k_range)
plt.savefig('silhouette_scores.png', dpi=300, bbox_inches='tight')
plt.close()

# 2. PCA visualization
print("Performing PCA and visualizing clusters...")
pca = PCA(n_components=2)
pca_result = pca.fit_transform(scaled_features)

# Print variance explained by PCA components
print(f"PCA Component 1 explains {pca.explained_variance_ratio_[0]:.2%} of variance")
print(f"PCA Component 2 explains {pca.explained_variance_ratio_[1]:.2%} of variance")
print(f"Total variance explained: {sum(pca.explained_variance_ratio_[:2]):.2%}")

# Create dataframe for plotting
pca_df = pd.DataFrame(data=pca_result, columns=['PC1', 'PC2'])
pca_df['cluster'] = cluster_labels
pca_df['stock_id'] = stock_ids.values

# Plot PCA
plt.figure(figsize=(12, 10))
sns.scatterplot(x='PC1', y='PC2', hue='cluster', data=pca_df, palette='viridis', 
                s=100, alpha=0.7, edgecolor='black', linewidth=2)
plt.title('Stock Clusters Based on ARMA-GARCH Features', fontsize=16)
plt.xlabel(f'Principal Component 1 ({pca.explained_variance_ratio_[0]:.2%} variance)', fontsize=12)
plt.ylabel(f'Principal Component 2 ({pca.explained_variance_ratio_[1]:.2%} variance)', fontsize=12)
plt.legend(title='Cluster', title_fontsize=12)
plt.grid(True, linestyle='--', alpha=0.3)
plt.savefig('pca_arma_garch_clusters.png', dpi=300, bbox_inches='tight')
plt.close()

# 3. Feature importance for clusters
print("\n--- CLUSTER CENTERS ---")
# Get cluster centers and calculate feature importance
cluster_centers = kmeans.cluster_centers_
feature_names = features_for_clustering.columns

# Print cluster centers
centers_df = pd.DataFrame(cluster_centers, columns=feature_names)
centers_df.index = [f'Cluster {i}' for i in range(best_k)]
print(centers_df)

# Heatmap of cluster centers
plt.figure(figsize=(16, 8))
sns.heatmap(centers_df, annot=True, cmap='coolwarm', linewidths=.5, fmt='.2f')
plt.title('ARMA-GARCH Feature Values by Cluster', fontsize=16)
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.savefig('cluster_centers_heatmap.png', dpi=300, bbox_inches='tight')
plt.close()

# 4. ARMA-GARCH specific features comparison
print("\n--- KEY ARMA-GARCH FEATURES BY CLUSTER ---")
# Create focused plots for the most relevant ARMA-GARCH features
arma_garch_features = [
    'serial_corr_pvalue', 'arch_effect_pvalue', 'acf_lag1', 
    'sq_acf_lag1', 'excess_kurtosis', 'vol_of_vol'
]

# Print key features by cluster
for feature in arma_garch_features:
    print(f"\nFeature: {feature}")
    feature_by_cluster = clustering_df.groupby('cluster')[feature].agg(['mean', 'std', 'min', 'max'])
    print(feature_by_cluster)

# Plot box plots for key ARMA-GARCH features by cluster
plt.figure(figsize=(16, 12))
for i, feature in enumerate(arma_garch_features):
    plt.subplot(2, 3, i+1)
    sns.boxplot(x='cluster', y=feature, data=clustering_df, palette='viridis')
    plt.title(f'{feature} by Cluster', fontsize=12)
    plt.grid(True, linestyle='--', alpha=0.3)
    
plt.tight_layout()
plt.savefig('arma_garch_features_boxplots.png', dpi=300, bbox_inches='tight')
plt.close()

# 5. Scatter plot matrix for ARMA-GARCH features
print("\nGenerating ARMA-GARCH feature relationship matrix...")
plt.figure(figsize=(15, 15))
plot_df = clustering_df[arma_garch_features + ['cluster', 'stock_id']]
scatter_matrix = sns.pairplot(plot_df, hue='cluster', palette='viridis', 
                             diag_kind='kde', plot_kws={'alpha': 0.6, 's': 60, 'edgecolor': 'white', 'linewidth': 0.5})
scatter_matrix.fig.suptitle('ARMA-GARCH Feature Relationships', y=1.02, fontsize=16)
plt.savefig('arma_garch_feature_matrix.png', dpi=300, bbox_inches='tight')
plt.close()

# 6. Best cluster for ARMA-GARCH identification
print("\n--- ARMA-GARCH SUITABILITY BY CLUSTER ---")
# Calculate an "ARMA-GARCH suitability score" for each cluster
# Lower p-values for serial correlation and ARCH effects are better
# Higher values for ACF, squared ACF, and kurtosis suggest better ARMA-GARCH fit

# First invert the p-values so higher is better
clustering_df['inv_serial_corr'] = 1 - clustering_df['serial_corr_pvalue']
clustering_df['inv_arch_effect'] = 1 - clustering_df['arch_effect_pvalue']

# Calculate mean values by cluster
cluster_scores = clustering_df.groupby('cluster').agg({
    'inv_serial_corr': 'mean',
    'inv_arch_effect': 'mean',
    'acf_lag1': lambda x: np.abs(x).mean(),  # Use absolute value as both negative and positive autocorrelation matter
    'sq_acf_lag1': lambda x: np.abs(x).mean(),
    'excess_kurtosis': 'mean',
    'vol_of_vol': 'mean'
})

# Print raw scores
print("Raw ARMA-GARCH suitability scores by cluster:")
print(cluster_scores)

# Normalize each feature to 0-1
normalized_scores = (cluster_scores - cluster_scores.min()) / (cluster_scores.max() - cluster_scores.min())
normalized_scores['arma_garch_score'] = normalized_scores.mean(axis=1)

# Print normalized scores
print("\nNormalized ARMA-GARCH suitability scores by cluster:")
print(normalized_scores)

# Plot the ARMA-GARCH suitability score by cluster
plt.figure(figsize=(12, 6))
sns.barplot(x=normalized_scores.index, y=normalized_scores['arma_garch_score'], palette='viridis')
plt.axhline(y=normalized_scores['arma_garch_score'].mean(), color='red', linestyle='--', label='Average Score')
plt.xticks([i for i in range(best_k)], [f'Cluster {i}' for i in range(best_k)])
plt.xlabel('Cluster', fontsize=12)
plt.ylabel('ARMA-GARCH Suitability Score', fontsize=12)
plt.title('Cluster Suitability for ARMA-GARCH Modeling', fontsize=14)
plt.legend()
plt.grid(True, linestyle='--', alpha=0.3)
plt.savefig('arma_garch_suitability_score.png', dpi=300, bbox_inches='tight')
plt.close()

# Print the best cluster for ARMA-GARCH modeling
best_cluster = normalized_scores['arma_garch_score'].idxmax()
print(f"\nThe most suitable cluster for ARMA-GARCH modeling is Cluster {best_cluster}")
print(f"ARMA-GARCH suitability score: {normalized_scores.loc[best_cluster, 'arma_garch_score']:.4f}")

# Print stocks in the best cluster
stocks_in_best_cluster = clustering_df[clustering_df['cluster'] == best_cluster]['stock_id'].tolist()
print(f"Stocks in this cluster: {stocks_in_best_cluster}")

# Print total number of stocks in the best cluster
print(f"Total number of stocks in Cluster {best_cluster}: {len(stocks_in_best_cluster)}")

# 7. Save the final clustered data
clustering_df.to_csv("arma_garch_clusters.csv")
print("\nAll visualizations have been saved to the current directory.")
print("Clustered data saved to arma_garch_clusters.csv")


--- FINDING OPTIMAL NUMBER OF CLUSTERS ---
For n_clusters = 2, the silhouette score is 0.2435379520170128
For n_clusters = 3, the silhouette score is 0.245789617196499
For n_clusters = 4, the silhouette score is 0.17138915106726688
For n_clusters = 5, the silhouette score is 0.15753062714040547
For n_clusters = 6, the silhouette score is 0.1585881966035648
For n_clusters = 7, the silhouette score is 0.13586813429552996
For n_clusters = 8, the silhouette score is 0.1700813591885071
For n_clusters = 9, the silhouette score is 0.13599067710025017
For n_clusters = 10, the silhouette score is 0.1521860875394228

Best number of clusters: 3

--- CLUSTER DISTRIBUTION ---
Cluster 0: 66 stocks
Cluster 1: 45 stocks
Cluster 2: 1 stocks

--- CREATING VISUALIZATIONS ---
Generating silhouette score plot...
Performing PCA and visualizing clusters...
PCA Component 1 explains 37.13% of variance
PCA Component 2 explains 13.21% of variance
Total variance explained: 50.34%

--- CLUSTER CENTERS ---
       

/var/folders/4g/_bw21mp523gbjm4t91xh0wp00000gn/T/ipykernel_80070/3351335454.py:136: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `x` variable to `hue` and set `legend=False` for the same effect.

  sns.boxplot(x='cluster', y=feature, data=clustering_df, palette='viridis')
/var/folders/4g/_bw21mp523gbjm4t91xh0wp00000gn/T/ipykernel_80070/3351335454.py:136: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `x` variable to `hue` and set `legend=False` for the same effect.

  sns.boxplot(x='cluster', y=feature, data=clustering_df, palette='viridis')
/var/folders/4g/_bw21mp523gbjm4t91xh0wp00000gn/T/ipykernel_80070/3351335454.py:136: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `x` variable to `hue` and set `legend=False` for the same effect.

  sns.boxplot(x='cluster', y=feature, data=cl


Generating ARMA-GARCH feature relationship matrix...

--- ARMA-GARCH SUITABILITY BY CLUSTER ---
Raw ARMA-GARCH suitability scores by cluster:
         inv_serial_corr  inv_arch_effect  acf_lag1  sq_acf_lag1  \
cluster                                                            
0               0.999997              1.0  0.121271     0.182128   
1               1.000000              1.0  0.198246     0.195714   
2               0.979272              1.0  0.000635     0.311566   

         excess_kurtosis  vol_of_vol  
cluster                               
0              27.407550    0.000110  
1              28.175342    0.000246  
2              23.841924    0.000039  

Normalized ARMA-GARCH suitability scores by cluster:
         inv_serial_corr  inv_arch_effect  acf_lag1  sq_acf_lag1  \
cluster                                                            
0               0.999877              NaN   0.61047     0.000000   
1               1.000000              NaN   1.00000     0.10496

/var/folders/4g/_bw21mp523gbjm4t91xh0wp00000gn/T/ipykernel_80070/3351335454.py:188: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `x` variable to `hue` and set `legend=False` for the same effect.

  sns.barplot(x=normalized_scores.index, y=normalized_scores['arma_garch_score'], palette='viridis')



The most suitable cluster for ARMA-GARCH modeling is Cluster 1
ARMA-GARCH suitability score: 0.8210
Stocks in this cluster: [94, 103, 0, 60, 9, 78, 8, 112, 102, 37, 27, 55, 126, 18, 100, 97, 87, 3, 11, 62, 72, 118, 33, 23, 58, 83, 114, 66, 6, 115, 82, 22, 40, 16, 4, 90, 80, 30, 98, 88, 17, 5, 75, 116, 38]
Total number of stocks in Cluster 1: 45

All visualizations have been saved to the current directory.
Clustered data saved to arma_garch_clusters.csv


<Figure size 1500x1500 with 0 Axes>

### ARMA-Garch fails to converge for different this data
### Next steps: Diagnostics were run to understand why it does not work
### AR - EWMA was deemed suitable and ran successfully

In [2]:
import numpy as np
import pandas as pd
import os
import glob
from arch import arch_model
from sklearn.metrics import mean_squared_error
import warnings
warnings.filterwarnings('ignore')

# Load the clustered data
clustered_data = pd.read_csv("arma_garch_clusters.csv")

# Directory containing parquet files
parquet_dir = "."

# Selected time_ids from your code
selected_time_ids = [5, 335, 697, 1107, 1326, 1640, 1938, 2178, 2479, 2833, 3113, 3469, 3877, 4138, 4460, 4743, 5101, 5407, 5729, 5978, 6325, 6643, 6930, 7245, 7629, 7921, 8441, 8718, 9091, 9367, 9819, 10100, 10491, 10781, 11070, 11431, 11764, 12030, 12348, 12661, 12965, 13318, 13579, 13868, 14176, 14452, 14795, 15038, 15418, 15728, 15989, 16404, 16704, 17029, 17305, 17606, 17914, 18184, 18502, 18836, 19157, 19488, 19747, 20135, 20460, 20827, 21145, 21507, 21891, 22249, 22622, 22922, 23220, 23531, 23863, 24224, 24707, 25059, 25347, 25636, 25924, 26307, 26603, 26944, 27240, 27595, 27964, 28379, 28801, 29171, 29590, 29941, 30313, 30624, 31047, 31322, 31609, 32083, 32463, 32767]

# Calculate QLIKE function
def calculate_qlike(actual_variance, predicted_variance):
    """Calculate QLIKE loss for volatility forecasts"""
    predicted_variance = np.maximum(predicted_variance, 1e-6)  # Avoid division by zero
    return actual_variance/predicted_variance - np.log(actual_variance/predicted_variance) - 1

# Results storage
results = {
    'cluster': [],
    'stock_id': [],
    'mse': [],
    'qlike': [],
    'converged': [],
    'model_type': []
}

# Process each cluster
for cluster in clustered_data['cluster'].unique():
    print(f"\nProcessing Cluster {cluster}")
    
    # Get stocks in this cluster
    cluster_stocks = clustered_data[clustered_data['cluster'] == cluster]['stock_id'].unique()
    
    # Process each stock in the cluster
    for stock_id in cluster_stocks:
        print(f"  Stock {stock_id}")
        
        try:
            # Load stock data from parquet file
            file_path = os.path.join(parquet_dir, f"stock_{stock_id}.parquet")
            if not os.path.exists(file_path):
                print(f"    File not found for stock {stock_id}")
                continue
                
            # Read the parquet file and filter for selected time_ids
            df = pd.read_parquet(file_path)
            df = df[df['time_id'].isin(selected_time_ids)]
            
            # Sort and get returns
            df = df.sort_values(['time_id', 'seconds_in_bucket'])
            returns = df['log_return'].dropna().values
            
            if len(returns) < 100:
                print(f"    Insufficient data for stock {stock_id}")
                continue
                
            # Only mild preprocessing - trim extreme outliers that can destabilize GARCH
            # Without changing the distribution characteristics too much
            lower_bound = np.percentile(returns, 0.5)
            upper_bound = np.percentile(returns, 99.5)
            returns_clean = np.clip(returns, lower_bound, upper_bound)
            
            # Split into train (80%) and test (20%)
            split = int(len(returns_clean) * 0.8)
            train = returns_clean[:split]
            test = returns_clean[split:]
            
            # List of model configurations to try
            model_configs = [
                # Standard AR(1)-GARCH(1,1) - try first
                {'name': 'AR(1)-GARCH(1,1)', 'spec': {'mean': 'AR', 'lags': 1, 'vol': 'GARCH', 'p': 1, 'q': 1}},
                # Try with constant mean if AR fails
                {'name': 'Constant-GARCH(1,1)', 'spec': {'mean': 'Constant', 'vol': 'GARCH', 'p': 1, 'q': 1}},
                # Try simpler ARCH model if GARCH fails
                {'name': 'AR(1)-ARCH(1)', 'spec': {'mean': 'AR', 'lags': 1, 'vol': 'ARCH', 'p': 1}},
            ]
            
            success = False
            best_result = None
            used_model = None
            
            # Try different model specifications
            for config in model_configs:
                if success:
                    break
                    
                try:
                    # Create model with current specification
                    model = arch_model(train, **config['spec'])
                    
                    # First try with default settings
                    try:
                        result = model.fit(disp='off', show_warning=False)
                        
                        # Check if converged with reasonable parameters
                        if result.convergence_status == 0:
                            success = True
                            best_result = result
                            used_model = config['name']
                            print(f"    Successfully fit {config['name']} model")
                            break
                    except:
                        # If default settings fail, try with more iterations
                        try:
                            result = model.fit(disp='off', options={'maxiter': 1000}, show_warning=False)
                            
                            # Check if converged with this attempt
                            if result.convergence_status == 0:
                                success = True
                                best_result = result
                                used_model = config['name']
                                print(f"    Successfully fit {config['name']} model with more iterations")
                                break
                        except:
                            # Continue to next model configuration
                            print(f"    Failed to fit {config['name']}")
                            continue
                            
                except Exception as e:
                    print(f"    Error with {config['name']} model: {str(e)}")
                    continue
            
            # If no model converged
            if not success:
                print(f"    All model configurations failed for stock {stock_id}")
                results['cluster'].append(cluster)
                results['stock_id'].append(stock_id)
                results['mse'].append(np.nan)
                results['qlike'].append(np.nan)
                results['converged'].append(False)
                results['model_type'].append("failed")
                continue
                
            # Make forecasts with the successful model
            try:
                forecasts = best_result.forecast(horizon=len(test))
                
                # Get predicted returns and volatility
                pred_returns = forecasts.mean.values[-1, :]
                pred_variance = forecasts.variance.values[-1, :]
                
                # Calculate metrics
                mse = mean_squared_error(test, pred_returns)
                
                # Use squared returns as proxy for actual variance
                actual_variance = test**2
                qlike = np.mean(calculate_qlike(actual_variance, pred_variance))
                
                # Store results
                results['cluster'].append(cluster)
                results['stock_id'].append(stock_id)
                results['mse'].append(mse)
                results['qlike'].append(qlike)
                results['converged'].append(True)
                results['model_type'].append(used_model)
                
            except Exception as e:
                print(f"    Error forecasting for stock {stock_id}: {str(e)}")
                results['cluster'].append(cluster)
                results['stock_id'].append(stock_id)
                results['mse'].append(np.nan)
                results['qlike'].append(np.nan)
                results['converged'].append(False)
                results['model_type'].append(f"{used_model}-forecast_failed")
            
        except Exception as e:
            print(f"    Error processing stock {stock_id}: {str(e)}")
            continue

# Convert results to DataFrame
results_df = pd.DataFrame(results)

# Print convergence statistics
converged_count = results_df['converged'].sum()
total_count = len(results_df)
print(f"\nConvergence Statistics: {converged_count}/{total_count} models converged ({converged_count/total_count*100:.1f}%)")

# Print model type statistics
model_stats = results_df.groupby('model_type').size().reset_index(name='count')
print("\nModel Type Statistics:")
print(model_stats)

# Calculate cluster-level metrics (only for converged models)
successful_results = results_df[results_df['converged']].copy()

if len(successful_results) > 0:
    # Calculate metrics by cluster
    cluster_summary = successful_results.groupby('cluster').agg({
        'mse': ['mean', 'std', 'count'],
        'qlike': ['mean', 'std', 'count']
    })
    
    # Add convergence rate
    convergence_by_cluster = pd.DataFrame({
        'total': results_df.groupby('cluster').size(),
        'converged': results_df[results_df['converged']].groupby('cluster').size()
    }).fillna(0)
    
    convergence_by_cluster['rate'] = convergence_by_cluster['converged'] / convergence_by_cluster['total']
    
    # Print summary
    print("\n--- Cluster Performance Summary ---")
    print(cluster_summary)
    
    print("\n--- Convergence by Cluster ---")
    print(convergence_by_cluster)
    
    # Find best clusters
    if not cluster_summary['mse']['mean'].isna().all():
        best_mse_cluster = cluster_summary['mse']['mean'].idxmin()
        print(f"\nBest cluster by MSE: Cluster {best_mse_cluster}")
    
    if not cluster_summary['qlike']['mean'].isna().all():
        best_qlike_cluster = cluster_summary['qlike']['mean'].idxmin()
        print(f"Best cluster by QLIKE: Cluster {best_qlike_cluster}")
else:
    print("\nNo models converged successfully for any stocks")

# Save results
results_df.to_csv("arma_garch_metrics_by_stock.csv", index=False)
if len(successful_results) > 0:
    cluster_summary.to_csv("arma_garch_metrics_by_cluster.csv")
    convergence_by_cluster.to_csv("arma_garch_convergence_by_cluster.csv")
    
    # Save successful results separately
    successful_results.to_csv("arma_garch_successful_models.csv", index=False)


Processing Cluster 1
  Stock 94
    Failed to fit AR(1)-GARCH(1,1)
    Failed to fit Constant-GARCH(1,1)
    Failed to fit AR(1)-ARCH(1)
    All model configurations failed for stock 94
  Stock 103
    Failed to fit AR(1)-GARCH(1,1)
    Failed to fit Constant-GARCH(1,1)
    Failed to fit AR(1)-ARCH(1)
    All model configurations failed for stock 103
  Stock 0
    Failed to fit AR(1)-GARCH(1,1)
    Failed to fit Constant-GARCH(1,1)
    Failed to fit AR(1)-ARCH(1)
    All model configurations failed for stock 0
  Stock 60
    Failed to fit AR(1)-GARCH(1,1)
    Failed to fit Constant-GARCH(1,1)
    Failed to fit AR(1)-ARCH(1)
    All model configurations failed for stock 60
  Stock 9
    Failed to fit AR(1)-GARCH(1,1)
    Failed to fit Constant-GARCH(1,1)
    Failed to fit AR(1)-ARCH(1)
    All model configurations failed for stock 9
  Stock 78
    Failed to fit AR(1)-GARCH(1,1)
    Failed to fit Constant-GARCH(1,1)
    Failed to fit AR(1)-ARCH(1)
    All model configurations failed for

#### Diagnostic analysis to understand why ARMA-Garch is not suitable:
- #### Inequality constraints are incompatible
- #### High percentage of zero returns 
- #### Extreme outliers

In [3]:
import numpy as np
import pandas as pd
import os
import glob
from arch import arch_model
from sklearn.metrics import mean_squared_error
import warnings
warnings.filterwarnings('ignore')

# Load the clustered data
clustered_data = pd.read_csv("arma_garch_clusters.csv")

# Directory containing parquet files
parquet_dir = "."

# Selected time_ids from your code
selected_time_ids = [5, 335, 697, 1107, 1326, 1640, 1938, 2178, 2479, 2833, 3113, 3469, 3877, 4138, 4460, 4743, 5101, 5407, 5729, 5978, 6325, 6643, 6930, 7245, 7629, 7921, 8441, 8718, 9091, 9367, 9819, 10100, 10491, 10781, 11070, 11431, 11764, 12030, 12348, 12661, 12965, 13318, 13579, 13868, 14176, 14452, 14795, 15038, 15418, 15728, 15989, 16404, 16704, 17029, 17305, 17606, 17914, 18184, 18502, 18836, 19157, 19488, 19747, 20135, 20460, 20827, 21145, 21507, 21891, 22249, 22622, 22922, 23220, 23531, 23863, 24224, 24707, 25059, 25347, 25636, 25924, 26307, 26603, 26944, 27240, 27595, 27964, 28379, 28801, 29171, 29590, 29941, 30313, 30624, 31047, 31322, 31609, 32083, 32463, 32767]

# Calculate QLIKE function
def calculate_qlike(actual_variance, predicted_variance):
    """Calculate QLIKE loss for volatility forecasts"""
    predicted_variance = np.maximum(predicted_variance, 1e-6)  # Avoid division by zero
    return actual_variance/predicted_variance - np.log(actual_variance/predicted_variance) - 1

# Function to diagnose data quality
def diagnose_returns(returns, stock_id):
    """Print diagnostic information about returns series"""
    print(f"\n----- DIAGNOSTICS FOR STOCK {stock_id} -----")
    print(f"Data shape: {returns.shape}")
    
    if len(returns) == 0:
        print("ERROR: No data found")
        return False
        
    print(f"Missing values: {np.isnan(returns).sum()}")
    print(f"Mean: {np.mean(returns)}")
    print(f"Std Dev: {np.std(returns)}")
    print(f"Min: {np.min(returns)}")
    print(f"Max: {np.max(returns)}")
    print(f"5% of zeros: {np.sum(returns == 0) / len(returns) * 100:.2f}%")
    
    # Print first few values
    print("First 10 values:", returns[:10])
    
    # Check for extreme outliers
    q1 = np.percentile(returns, 25)
    q3 = np.percentile(returns, 75)
    iqr = q3 - q1
    outliers = np.sum((returns < q1 - 3 * iqr) | (returns > q3 + 3 * iqr))
    print(f"Extreme outliers: {outliers} ({outliers/len(returns)*100:.2f}%)")
    
    # Check for clustering of volatility (ARCH effects)
    squared_returns = returns**2
    lag_1_corr = np.corrcoef(squared_returns[:-1], squared_returns[1:])[0, 1]
    print(f"Volatility clustering (lag-1 correlation of squared returns): {lag_1_corr:.4f}")
    
    return True

# Results storage
results = {
    'cluster': [],
    'stock_id': [],
    'mse': [],
    'qlike': [],
    'converged': [],
    'model_type': []
}

# Process each cluster
print("\n===== STARTING DATA INSPECTION =====")
# Choose a sample stock for detailed diagnostics
sample_stocks = []
for cluster in clustered_data['cluster'].unique():
    cluster_stocks = clustered_data[clustered_data['cluster'] == cluster]['stock_id'].unique()
    if len(cluster_stocks) > 0:
        sample_stocks.append(cluster_stocks[0])

# Inspect a few sample stocks in detail
for stock_id in sample_stocks[:3]:  # Look at first 3 stocks
    try:
        file_path = os.path.join(parquet_dir, f"stock_{stock_id}.parquet")
        if os.path.exists(file_path):
            df = pd.read_parquet(file_path)
            df = df[df['time_id'].isin(selected_time_ids)]
            
            print(f"\nSample stock {stock_id} data columns: {df.columns.tolist()}")
            print(f"Sample rows:")
            print(df.head(3))
            
            # Check time_id distribution
            print(f"Found {len(df['time_id'].unique())} unique time_ids out of {len(selected_time_ids)} expected")
            
            if 'log_return' in df.columns:
                returns = df['log_return'].dropna().values
                diagnose_returns(returns, stock_id)
            else:
                print(f"WARNING: 'log_return' column not found in stock {stock_id}")
    except Exception as e:
        print(f"Error inspecting stock {stock_id}: {str(e)}")

print("\n===== STARTING MODEL FITTING =====")

# Set a flag to use simple models that are more likely to converge
# This uses much simpler models that should converge more easily
USE_SIMPLIFIED_MODELS = True

# Process each cluster
for cluster in clustered_data['cluster'].unique()[:3]:  # Start with just first 3 clusters
    print(f"\nProcessing Cluster {cluster}")
    
    # Get stocks in this cluster
    cluster_stocks = clustered_data[clustered_data['cluster'] == cluster]['stock_id'].unique()
    
    # Process each stock in the cluster
    for stock_id in cluster_stocks[:5]:  # Process just 5 stocks per cluster initially
        print(f"  Stock {stock_id}")
        
        try:
            # Load stock data from parquet file
            file_path = os.path.join(parquet_dir, f"stock_{stock_id}.parquet")
            if not os.path.exists(file_path):
                print(f"    File not found for stock {stock_id}")
                continue
                
            # Read the parquet file and filter for selected time_ids
            df = pd.read_parquet(file_path)
            df = df[df['time_id'].isin(selected_time_ids)]
            
            # Sort and get returns
            df = df.sort_values(['time_id', 'seconds_in_bucket'])
            returns = df['log_return'].dropna().values
            
            if len(returns) < 50:  # Reduced minimum length
                print(f"    Insufficient data for stock {stock_id}")
                continue
            
            # Data diagnostics for the first stock in each cluster
            if stock_id == cluster_stocks[0]:
                diagnose_returns(returns, stock_id)
                
            # Split into train (80%) and test (20%)
            split = int(len(returns) * 0.8)
            train = returns[:split]
            test = returns[split:]
            
            # Try to fit a VERY simple model first
            success = False
            best_result = None
            used_model = None
            
            # List of model configurations to try - ordered from simplest to more complex
            if USE_SIMPLIFIED_MODELS:
                model_configs = [
                    # Pure ARCH model with constant mean - extremely simple
                    {'name': 'Constant-ARCH(1)', 'spec': {'mean': 'Constant', 'vol': 'ARCH', 'p': 1}},
                    # Standard GARCH model with constant mean
                    {'name': 'Constant-GARCH(1,1)', 'spec': {'mean': 'Constant', 'vol': 'GARCH', 'p': 1, 'q': 1}},
                    # AR(1) mean model with ARCH volatility
                    {'name': 'AR(1)-ARCH(1)', 'spec': {'mean': 'AR', 'lags': 1, 'vol': 'ARCH', 'p': 1}},
                ]
            else:
                # Original model configurations
                model_configs = [
                    {'name': 'AR(1)-GARCH(1,1)', 'spec': {'mean': 'AR', 'lags': 1, 'vol': 'GARCH', 'p': 1, 'q': 1}},
                    {'name': 'Constant-GARCH(1,1)', 'spec': {'mean': 'Constant', 'vol': 'GARCH', 'p': 1, 'q': 1}},
                    {'name': 'AR(1)-ARCH(1)', 'spec': {'mean': 'AR', 'lags': 1, 'vol': 'ARCH', 'p': 1}},
                ]
            
            # Try different model specifications
            for config in model_configs:
                if success:
                    break
                    
                try:
                    print(f"    Trying {config['name']} model...")
                    # Create model with current specification
                    model = arch_model(train, **config['spec'])
                    
                    # Turn on detailed display to see what's happening during convergence
                    result = model.fit(disp='off', show_warning=False)
                    convergence_status = result.convergence_status
                    
                    print(f"    {config['name']} convergence status: {convergence_status}")
                    
                    # Check convergence
                    if convergence_status == 0:
                        success = True
                        best_result = result
                        used_model = config['name']
                        print(f"    Successfully fit {config['name']} model")
                        break
                    else:
                        print(f"    Model didn't converge. Trying next configuration.")
                        
                except Exception as e:
                    print(f"    Error with {config['name']} model: {str(e)}")
                    continue
            
            # If no model converged
            if not success:
                print(f"    All model configurations failed for stock {stock_id}")
                results['cluster'].append(cluster)
                results['stock_id'].append(stock_id)
                results['mse'].append(np.nan)
                results['qlike'].append(np.nan)
                results['converged'].append(False)
                results['model_type'].append("failed")
                continue
                
            # Make forecasts with the successful model
            try:
                forecasts = best_result.forecast(horizon=len(test))
                
                # Get predicted returns and volatility
                pred_returns = forecasts.mean.values[-1, :]
                pred_variance = forecasts.variance.values[-1, :]
                
                # Calculate metrics
                mse = mean_squared_error(test, pred_returns)
                
                # Use squared returns as proxy for actual variance
                actual_variance = test**2
                qlike = np.mean(calculate_qlike(actual_variance, pred_variance))
                
                # Store results
                results['cluster'].append(cluster)
                results['stock_id'].append(stock_id)
                results['mse'].append(mse)
                results['qlike'].append(qlike)
                results['converged'].append(True)
                results['model_type'].append(used_model)
                
            except Exception as e:
                print(f"    Error forecasting for stock {stock_id}: {str(e)}")
                results['cluster'].append(cluster)
                results['stock_id'].append(stock_id)
                results['mse'].append(np.nan)
                results['qlike'].append(np.nan)
                results['converged'].append(False)
                results['model_type'].append(f"{used_model}-forecast_failed")
            
        except Exception as e:
            print(f"    Error processing stock {stock_id}: {str(e)}")
            continue

# Convert results to DataFrame
results_df = pd.DataFrame(results)

# Print convergence statistics
if len(results_df) > 0:
    converged_count = results_df['converged'].sum()
    total_count = len(results_df)
    print(f"\nConvergence Statistics: {converged_count}/{total_count} models converged ({converged_count/total_count*100:.1f}%)")

    # Print model type statistics
    model_stats = results_df.groupby('model_type').size().reset_index(name='count')
    print("\nModel Type Statistics:")
    print(model_stats)

    # Calculate cluster-level metrics (only for converged models)
    successful_results = results_df[results_df['converged']].copy()

    if len(successful_results) > 0:
        # Calculate metrics by cluster
        cluster_summary = successful_results.groupby('cluster').agg({
            'mse': ['mean', 'std', 'count'],
            'qlike': ['mean', 'std', 'count']
        })
        
        # Add convergence rate
        convergence_by_cluster = pd.DataFrame({
            'total': results_df.groupby('cluster').size(),
            'converged': results_df[results_df['converged']].groupby('cluster').size()
        }).fillna(0)
        
        convergence_by_cluster['rate'] = convergence_by_cluster['converged'] / convergence_by_cluster['total']
        
        # Print summary
        print("\n--- Cluster Performance Summary ---")
        print(cluster_summary)
        
        print("\n--- Convergence by Cluster ---")
        print(convergence_by_cluster)
        
        # Find best clusters
        if not cluster_summary['mse']['mean'].isna().all():
            best_mse_cluster = cluster_summary['mse']['mean'].idxmin()
            print(f"\nBest cluster by MSE: Cluster {best_mse_cluster}")
        
        if not cluster_summary['qlike']['mean'].isna().all():
            best_qlike_cluster = cluster_summary['qlike']['mean'].idxmin()
            print(f"Best cluster by QLIKE: Cluster {best_qlike_cluster}")
    else:
        print("\nNo models converged successfully for any stocks")
else:
    print("\nNo results were collected.")

print("\n===== RECOMMENDATIONS =====")
print("""
Based on the diagnostics, here are strategies to try:
1. Use simpler models: Start with ARCH(1) with constant mean
2. If volatility clustering is weak, consider just using AR models without GARCH
3. For extreme data, try using a Student's t distribution instead of normal
4. Reduce the data requirements - work with fewer time periods
5. Consider using a different volatility modeling approach like RiskMetrics (EWMA)
""")

# Save diagnostic version results
if len(results_df) > 0:
    results_df.to_csv("arma_garch_diagnostics.csv", index=False)


===== STARTING DATA INSPECTION =====

Sample stock 94 data columns: ['time_id', 'seconds_in_bucket', 'bid_price1', 'ask_price1', 'bid_price2', 'ask_price2', 'bid_size1', 'ask_size1', 'bid_size2', 'ask_size2', 'stock_id', 'WAP', 'BidAskSpread', 'log_return', 'time_bucket', 'volatility']
Sample rows:
   time_id  seconds_in_bucket  bid_price1  ask_price1  bid_price2  ask_price2  \
0        5                  0    1.000301    1.000903    1.000129    1.000989   
1        5                  3    1.000301    1.000903    1.000129    1.000989   
2        5                  4    1.000301    1.000903    1.000043    1.000989   

   bid_size1  ask_size1  bid_size2  ask_size2  stock_id       WAP  \
0        101         60        101         10        94  1.000679   
1        101         60        101         10        94  1.000679   
2        101         71          1         10        94  1.000655   

   BidAskSpread  log_return  time_bucket  volatility  
0      0.000602         NaN            0  

#### AR-EWMA runs better on the current data regardless of clusters and stocks (no covergence issues). Almost all stock in clusters have convergence issues when using ARMA - Garch.

In [4]:
import numpy as np
import pandas as pd
import os
import glob
from arch import arch_model
from sklearn.metrics import mean_squared_error
import warnings
warnings.filterwarnings('ignore')

# Load the clustered data
clustered_data = pd.read_csv("arma_garch_clusters.csv")

# Directory containing parquet files
parquet_dir = "."

# Selected time_ids from your code
selected_time_ids = [5, 335, 697, 1107, 1326, 1640, 1938, 2178, 2479, 2833, 3113, 3469, 3877, 4138, 4460, 4743, 5101, 5407, 5729, 5978, 6325, 6643, 6930, 7245, 7629, 7921, 8441, 8718, 9091, 9367, 9819, 10100, 10491, 10781, 11070, 11431, 11764, 12030, 12348, 12661, 12965, 13318, 13579, 13868, 14176, 14452, 14795, 15038, 15418, 15728, 15989, 16404, 16704, 17029, 17305, 17606, 17914, 18184, 18502, 18836, 19157, 19488, 19747, 20135, 20460, 20827, 21145, 21507, 21891, 22249, 22622, 22922, 23220, 23531, 23863, 24224, 24707, 25059, 25347, 25636, 25924, 26307, 26603, 26944, 27240, 27595, 27964, 28379, 28801, 29171, 29590, 29941, 30313, 30624, 31047, 31322, 31609, 32083, 32463, 32767]

# Calculate QLIKE function
def calculate_qlike(actual_variance, predicted_variance):
    """Calculate QLIKE loss for volatility forecasts"""
    predicted_variance = np.maximum(predicted_variance, 1e-6)  # Avoid division by zero
    return actual_variance/predicted_variance - np.log(actual_variance/predicted_variance) - 1

# Function to compute EWMA volatility
def calculate_ewma_variance(returns, lambda_param=0.94):
    """
    Calculate EWMA (Exponentially Weighted Moving Average) variance
    This is similar to RiskMetrics approach and more robust than GARCH for noisy data
    """
    # Initialize variance with sample variance for first observation
    n = len(returns)
    variance = np.zeros(n)
    
    # Use first 20 observations to initialize
    if n > 20:
        variance[0] = np.var(returns[:20])
    else:
        variance[0] = returns[0]**2
    
    # Calculate EWMA variance
    for t in range(1, n):
        variance[t] = lambda_param * variance[t-1] + (1 - lambda_param) * returns[t-1]**2
    
    return variance

# Function to fit a simple AR model using OLS
def fit_ar_model(returns, lag=1):
    """
    Fit a simple AR model using OLS regression
    Returns parameters and fitted values
    """
    y = returns[lag:]
    X = np.column_stack([np.ones(len(y)), returns[:-lag]])
    
    # OLS estimation of parameters
    try:
        beta = np.linalg.inv(X.T @ X) @ X.T @ y
        fitted = X @ beta
        return beta, fitted
    except:
        # If matrix is singular, return simple mean model
        mean_return = np.mean(returns)
        return np.array([mean_return, 0]), np.full(len(y), mean_return)

# Results storage
results = {
    'cluster': [],
    'stock_id': [],
    'mse': [],
    'qlike': [],
    'model_type': []
}

# Process each cluster
for cluster in clustered_data['cluster'].unique():
    print(f"\nProcessing Cluster {cluster}")
    
    # Get stocks in this cluster
    cluster_stocks = clustered_data[clustered_data['cluster'] == cluster]['stock_id'].unique()
    
    # Process each stock in the cluster
    for stock_id in cluster_stocks:
        print(f"  Stock {stock_id}")
        
        try:
            # Load stock data from parquet file
            file_path = os.path.join(parquet_dir, f"stock_{stock_id}.parquet")
            if not os.path.exists(file_path):
                print(f"    File not found for stock {stock_id}")
                continue
                
            # Read the parquet file and filter for selected time_ids
            df = pd.read_parquet(file_path)
            df = df[df['time_id'].isin(selected_time_ids)]
            
            # Sort and get returns
            df = df.sort_values(['time_id', 'seconds_in_bucket'])
            returns = df['log_return'].dropna().values
            
            if len(returns) < 50:  # We need some history for EWMA
                print(f"    Insufficient data for stock {stock_id}")
                continue
            
            # Handle outliers by winsorizing at 3 standard deviations
            std_dev = np.std(returns)
            mean_return = np.mean(returns)
            upper_bound = mean_return + 3 * std_dev
            lower_bound = mean_return - 3 * std_dev
            returns_clean = np.clip(returns, lower_bound, upper_bound)
            
            # Split into train (80%) and test (20%)
            split = int(len(returns_clean) * 0.8)
            train = returns_clean[:split]
            test = returns_clean[split:]
            
            # 1. Fit AR(1) model for returns using OLS (much more robust than MLE)
            ar_params, fitted_returns = fit_ar_model(train, lag=1)
            
            # 2. Calculate EWMA variance on the training data
            train_variance = calculate_ewma_variance(train)
            
            # Forecast returns for test period using AR(1) model
            test_X = np.column_stack([np.ones(len(test)), np.append(train[-1], test[:-1])])
            pred_returns = test_X @ ar_params
            
            # Forecast variance for test period using EWMA
            # First we extend the train variance series
            full_variance = np.zeros(len(train) + len(test))
            full_variance[:len(train)] = train_variance
            
            # Get the last train variance as starting point
            last_train_var = train_variance[-1]
            lambda_param = 0.94  # Standard RiskMetrics value
            
            # Start with the last training variance and forecast forward
            pred_variance = np.zeros(len(test))
            curr_var = last_train_var
            
            for i in range(len(test)):
                # If we have some test data calculated, use it, otherwise use training data
                if i > 0:
                    prev_return = test[i-1]
                else:
                    prev_return = train[-1]
                
                # EWMA update formula
                curr_var = lambda_param * curr_var + (1 - lambda_param) * prev_return**2
                pred_variance[i] = curr_var
            
            # Calculate metrics
            mse = mean_squared_error(test, pred_returns)
            
            # Use squared returns as proxy for actual variance
            actual_variance = test**2
            qlike = np.mean(calculate_qlike(actual_variance, pred_variance))
            
            # Store results
            results['cluster'].append(cluster)
            results['stock_id'].append(stock_id)
            results['mse'].append(mse)
            results['qlike'].append(qlike)
            results['model_type'].append("AR(1)-EWMA")
            
            print(f"    Successfully modeled using AR(1)-EWMA")
            
        except Exception as e:
            print(f"    Error processing stock {stock_id}: {str(e)}")
            continue

# Convert results to DataFrame
results_df = pd.DataFrame(results)

print(f"\nSuccessfully modeled {len(results_df)} stocks")

# Calculate cluster-level metrics
if len(results_df) > 0:
    cluster_summary = results_df.groupby('cluster').agg({
        'mse': ['mean', 'std', 'count'],
        'qlike': ['mean', 'std', 'count']
    })
    
    # Print summary
    print("\n--- Cluster Performance Summary ---")
    print(cluster_summary)
    
    # Find best clusters
    if not cluster_summary['mse']['mean'].isna().all():
        best_mse_cluster = cluster_summary['mse']['mean'].idxmin()
        print(f"\nBest cluster by MSE: Cluster {best_mse_cluster}")
    
    if not cluster_summary['qlike']['mean'].isna().all():
        best_qlike_cluster = cluster_summary['qlike']['mean'].idxmin()
        print(f"Best cluster by QLIKE: Cluster {best_qlike_cluster}")
else:
    print("\nNo models successfully computed metrics")

# Save results
results_df.to_csv("ar_ewma_metrics_by_stock.csv", index=False)
if len(results_df) > 0:
    cluster_summary.to_csv("ar_ewma_metrics_by_cluster.csv")


Processing Cluster 1
  Stock 94
    Successfully modeled using AR(1)-EWMA
  Stock 103
    Successfully modeled using AR(1)-EWMA
  Stock 0
    Successfully modeled using AR(1)-EWMA
  Stock 60
    Successfully modeled using AR(1)-EWMA
  Stock 9
    Successfully modeled using AR(1)-EWMA
  Stock 78
    Successfully modeled using AR(1)-EWMA
  Stock 8
    Successfully modeled using AR(1)-EWMA
  Stock 112
    Successfully modeled using AR(1)-EWMA
  Stock 102
    Successfully modeled using AR(1)-EWMA
  Stock 37
    Successfully modeled using AR(1)-EWMA
  Stock 27
    Successfully modeled using AR(1)-EWMA
  Stock 55
    Successfully modeled using AR(1)-EWMA
  Stock 126
    Successfully modeled using AR(1)-EWMA
  Stock 18
    Successfully modeled using AR(1)-EWMA
  Stock 100
    Successfully modeled using AR(1)-EWMA
  Stock 97
    Successfully modeled using AR(1)-EWMA
  Stock 87
    Successfully modeled using AR(1)-EWMA
  Stock 3
    Successfully modeled using AR(1)-EWMA
  Stock 11
    Successf

You need to print out results and also refer to papers and articles and then train model on the last few clusters and then have performance metrics